<h1 align="center"> Computation for Physicists </h1>
<h2 align="center"> <em> Introduction to Scientific Computing</em> </h2>
<h2 align="center" > <a href="mailto:duan@unm.edu">Dr. Duan</a> (UNM) </h2>

# Homework
- The $n$th Fibonacci number is defined as $F_n = F_{n-1} + F_{n-2}$ with $F_0=0$ and $F_1=1$. Define a function to compute an arbitrary Fibonacci number.
- Document the function properly and check if the input is valid.
- Store the computed Fibonacci numbers for reuse.

In [ ]:
from homework.hw3 import * # import everything in homework/hw3.py

for n in range(15):
        print(Fibonacci(n), end=", ")
print('...')

In [ ]:
Fibonacci(-1)

In [ ]:
Fibonacci?

In [ ]:
_fib_seq? 

- By default, a variable with a name starting with an underscore is not imported by `import *` to avoid name conflict.

In [ ]:
import homework.hw3 as hw3
hw3._fib_seq 

- The statements in the module are executed only once at the first import.
- Use `importlib.reload()` to have a fresh reload of the module.

# Numbers in a Computer
- Most computers store numbers in the binary (base 2) format.
- An $n$-bit unsigned binary integer can be written as
$$(d_{n-1}\cdots d_1 d_0)_2 = d_{n-1}\cdot 2^{n-1}  + \cdots + d_{1}\cdot 2 + d_0,$$
where $d_i\in\{0,1\}$. 

In [ ]:
0b110 == 4+2, 0b1000 == 2**3, 0b111 == 2**3 - 1 # binary format starts with 0b

- See [NumPy documentation](https://numpy.org/devdocs/user/basics.types.html) for the list of its supported data types.
- Some of the integer types supported by NumPy:
    - `numpy.uint32`: unsigned 32-bit integer, $0,1,2,\ldots,2^{32}-1$.
    - `numpy.uint64`: unsigned 64-bit integer, $0,1,2,\ldots,2^{64}-1$.
    - `numpy.int32`: signed 32-bit integer, $-2^{31}$ to $2^{31}-1$.
    - `numpy.int64`: signed 64-bit integer, $-2^{63}$ to $2^{63}-1$.

In [ ]:
import numpy as np
a = np.array([2], dtype=np.int32) # use dtype to specify data type
2**31, a**31, (-a)**31 # be careful with the overflow

In [ ]:
np.iinfo(np.int32)

- Floating-point (FP) numbers are written as
$$x = \pm \left(d_0 + \frac{d_1}{2} + \frac{d_2}{2^2} + \cdots + \frac{d_{p-1}}{2^{p-1}}\right)\times 2^E,$$
where the integral numbers $p$ and $E$ are the  _precision_ and _exponent_, respectively. $d_0d_1\cdots d_{p-1}$ is called the _mantissa_, and $d_1\cdots d_{p-1}$ is called the _fraction_.

- An example of the layout of a 32-bit FP number:
    - $-126\leq E \leq 127$ with $-126$ and $127$ represented by all 0's and all 1's in the exponent bits, respectively.
    - A _normalized number_ always has $d_0=1$ which is not stored.
    
![32-bit floating point](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Float_example.svg/800px-Float_example.svg.png) 
(Figure Credit: WikiMedia)
    

In [ ]:
(1+1/4)*2**-3 # E=0 when the exponent is 0b01111111 (127)

- Some of the FP types supported by Numpy:
    - `numpy.float32`: single precision (SP) or 32-bit FP number ($p=24$, $-126\leq E\leq127$).
    - `numpy.float64` (default): double precision (DP) or 64-bit FP number ($p=53$, $-1022\leq E\leq1023$).
    - `numpy.complex64`: two SP numbers.
    - `numpy.complex128`: two DP numbers.
- Numpy also defines `numpy.inf` (infinity) and `numpy.nan` (not a number).

In [ ]:
np.array([2**127], dtype=np.float32) *2, np.array([0.])/0.

In [ ]:
np.finfo(np.float32)

In [ ]:
np.finfo(np.float64)

- Don't compare FP numbers with `==`.

In [ ]:
0.3/3 == 0.1 

In [ ]:
print(f'{0.3:.17f}, {0.1:.17f}')

See, e.g., [https://realpython.com/python-formatted-output](https://realpython.com/python-formatted-output) for a tutorial on formatting a Python string.

In [ ]:
abs(0.3/3 - 0.1) <= 1e-5

# Rouding Error

- A FP number has a finite precision or number of significant digits ($\sim 7$ for SP and $\sim16$ for DP).

In [ ]:
eps_single = 2**-23 # machine precision for single
eps_single

In [ ]:
a = np.array([1.], dtype=np.float32)
a + eps_single == a, a + eps_single/2 == a

In [ ]:
eps_double = 2**-52 # machine precision for double
b = np.array([1.])
eps_double, b + eps_double == b, b + eps_double/2 == b


- Subtraction of two similar numbers can cause loss of significant digits, e.g., 1.0001 - 1.0000 = 1e-4.

In [ ]:
np.array([1. + 2*eps_double]) - np.array([1. + 1.5*eps_double]) 

# Error Analysis
Many of the numerical examples and algorithms are from [Scientific Computing: An Introductory Survey](https://www.amazon.com/Scientific-Computing-Introductory-Survey-Revised/dp/1611975573)
![Scientific Computing](http://heath.cs.illinois.edu/scicomp/images/CL80large.jpg)

> I’ve learned that, in the description of Nature, one has to tolerate approximations, and that work with approximations can be interesting and can sometimes be beautiful. -- P.A.M. Dirac

- Example: Compute the surface area of Earth.

In [ ]:
R_Earth = 6378e3 # Earth radius in m (Google)
print(f"The surface area of Earth is approximately {4*np.pi*R_Earth:.4g} m^2.")

## Numerical Approach
- Compute an alternative, approximate problem: Approximating the true value $y$ by an algorithm $\hat{f}(x)$ that can be more easily computed.
- Use approximate inputs: Replacing $x$ by $\hat{x}$ with a finite precision. This introduces the data error $\Delta x=\hat{x}-x$.
- The final result $\hat{y}=\hat{f}(\hat{x})$ has an _absolute error_ (uncertainty) $\Delta y =\hat{y}-y$.
- Numerical computations usually control the _relative error_ which is defined as $\Delta y/y\approx \Delta y/ \hat{y}$. 

- Total error = <span style='color:blue'> computational error </span> + <span style='color:red'> propagated data error </span>
$$\hat{f}(\hat{x}) - f(x) = \color{blue}{[\hat{f}(\hat{x}) - f(\hat{x})]} + \color{red}{[f(\hat{x}) - f(x)]},$$
where $f$ is the exact algorithm approximated by $\hat{f}$.
- Computational error = truncation error + rounding error.
    - Rounding error: due to the limited precision.
    - Truncation error: due to the algorithm.

In [ ]:
def mycos(x, tol=1e-6):
    # x = x % (2.0*pi) # move x to [0, 2*pi)
    res = 0. # result
    term = 1. # 1st term in the Taylor series
    n = 0 # power of x for the term
    while abs(term) >= tol: 
        res += term # add the term to the result
        term *= -x*x / ((n+1)*(n+2)) # compute the new term
        n += 2 # skip the odd power of x    
    # res += term # add the last term
    print(f"(-1)^{n//2} x^{n} / {n}! = {term}")
    return res

Is `tol` the tolerance of the absolute error or the relative error?

In [ ]:
np.cos(3.) - mycos(3.) # dominated by truncation error

In [ ]:
np.cos(100.) - mycos(100.) # dominated by rounding error

# Homework Problem
- The standard deviation of a dataset $\{x_1, x_2, \cdots, x_n\}$ is defined as
$$\sigma = \sqrt{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2},$$
where $\bar{x}$ is the average value of $x_i$. It is straightforward to show that an equivalent definition of the standard deviation is
$$\sigma = \sqrt{\left(\frac{1}{n}\sum_{i=1}^n x_i^2\right)- \bar{x}^2}.$$
Which of the two algorithms is potentially faster to compute, and which is more accurate?

- Verify your conjecture with some example(s). You may find the NumPy functions `numpy.random.random()`, `numpy.sum()`, `numpy.mean()`, and `numpy.std()` useful.